# MatchingUsers: this is a test

In [ ]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from Utils import TransantiagoConstants
import EvasionBuilder

In [ ]:
processed_evasion = EvasionBuilder.runCompleteProcess()

* Processing evasion-ddbb before merging

In [ ]:
evasion_paradero = processed_evasion[processed_evasion['TP']=='P']
evasion_paradero_first = evasion_paradero[evasion_paradero['N_PUERTA']==1]
print(type(processed_evasion['SERVICIO'][0])) #should be a string

In [ ]:
print('Number of rows in complete evasion database is: ' + str(len(processed_evasion.index)))
print('Number of rows in evasion in paradero database is: ' + str(len(evasion_paradero.index)))
print('Number of rows in evasion in paradero by first door database is: ' + str(len(evasion_paradero_first.index)))

* Filtering dates not in common dates

In [ ]:
common_dates = TransantiagoConstants.common_dates
common_dates_timestamp = [pd.to_datetime(x) for x in common_dates]
common_dates_evasion = evasion_paradero_first[evasion_paradero_first['FECHA'].isin(common_dates_timestamp)]
len(common_dates_evasion.index) #Remember to store the number.

* Filtering for one specific common date (2017-04-11) and analyze it to refactor and debug

In [ ]:
example = common_dates_evasion[common_dates_evasion['FECHA']==pd.to_datetime('2017-04-11')]

In [ ]:
len(example.index)

In [ ]:
from RunSilentlyDailyEtapasBuilder import RunSilentlyDailyEtapasBuilderClass

In [ ]:
date = '2017-04-11'
etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
processed_sorted_df = etapas_builder.runProcessedProcess()

In [ ]:
len(processed_sorted_df.index)

In [ ]:
#processed_sorted_df = processed_sorted_df[processed_sorted_df['diferencia_tiempo_secs']<=20]

* Processing processed_sorted_df before reducing complexity

In [ ]:
processed_sorted_df['sitio_subida'] = processed_sorted_df['sitio_subida'].str.replace("-", "")
processed_sorted_df['sitio_subida'] = processed_sorted_df['sitio_subida'].str.replace(" ", "")

In [ ]:
processed_sorted_df['servicio_subida'] =  processed_sorted_df['servicio_subida'].str.replace('T','')
processed_sorted_df['servicio_subida'] =  processed_sorted_df['servicio_subida'].str.split(' ').str[0]

In [ ]:
trx_servicios = processed_sorted_df['servicio_subida'].unique().tolist()
for servicio in trx_servicios:
    if 'y' in servicio:
        print(servicio)

In [ ]:
for servicio in trx_servicios:
    if 'Y' in servicio:
        print(servicio)

* First: filter etapas by patentes and servicios surveyed in evasion ddbb so to reduce search space

In [ ]:
patentes = example['PATENTE'].unique().tolist()
servicios = example['SERVICIO'].unique().tolist()

In [ ]:
clean_sorted_df = processed_sorted_df[(processed_sorted_df['sitio_subida'].isin(patentes))&(processed_sorted_df['servicio_subida'].isin(servicios))]

In [ ]:
clean_sorted_df = clean_sorted_df.reset_index(drop=True)

In [ ]:
len(clean_sorted_df.index)

### STARTING POINT FOR METHODOLOGY

In [ ]:
past_servicio = ''
past_patente = ''
past_time = pd.to_datetime('')
id_exp = 1
clean_sorted_df['idExpedicion'] = ''

for index,row in clean_sorted_df.iterrows():
    actual_servicio = row['servicio_subida']
    actual_patente = row['sitio_subida']
    actual_time = row['t_subida']
    
    #THRESHOLD TO CONSIDER A ROW IN A DIFFERENT EXPEDITION IS 15 MINUTES. THIS IS AD-HOC, BUT SHOULD NOT HAVE EFFECT ON FINAL RESULT
    if((past_servicio==actual_servicio)&(actual_time - past_time <= (pd.to_datetime('2017-04-11 00:15:00')-pd.to_datetime('2017-04-11 00:00:00')))):
        clean_sorted_df.loc[index,'idExpedicion'] = id_exp

    elif((past_servicio==actual_servicio)&(actual_time - past_time > (pd.to_datetime('2017-04-11 00:15:00')-pd.to_datetime('2017-04-11 00:00:00')))):
        id_exp = id_exp + 1
        clean_sorted_df.loc[index,'idExpedicion'] = id_exp
        
    elif((past_servicio!=actual_servicio)):
        id_exp = 1
        clean_sorted_df.loc[index,'idExpedicion'] = id_exp
        

    past_servicio = actual_servicio
    past_patente = actual_patente
    past_time = actual_time
    

In [ ]:
clean_sorted_df.head()

In [ ]:
len(clean_sorted_df.index)

In [ ]:
f = {'t_subida':['min', 'max', 'count'], 'diferencia_tiempo_secs':['mean']}
grouped_clean_sorted_df = clean_sorted_df.groupby(['sitio_subida','servicio_subida','idExpedicion','par_subida']).agg(f)
grouped_clean_sorted_df = grouped_clean_sorted_df.reset_index()
grouped_clean_sorted_df.columns = [''.join(col).strip() for col in grouped_clean_sorted_df.columns.values]
grouped_clean_sorted_df = grouped_clean_sorted_df.sort_values(by=['sitio_subida', 'servicio_subida', 'idExpedicion', 't_subidamin'], ascending=[True, True, True, True])
grouped_clean_sorted_df = grouped_clean_sorted_df.reset_index(drop=True)

In [ ]:
grouped_clean_sorted_df.head()

In [ ]:
len(grouped_clean_sorted_df.index)

In [ ]:
i = 0
past_plate = '-'
past_service = '-'

past_par_subida = ['']

past_initial_hour = pd.to_datetime('2017-04-11 00:00:00')
past_end_hour = pd.to_datetime('2017-04-11 00:00:00')
past_id_expedicion = '-'

grouped_clean_sorted_df['start_cut']=''
grouped_clean_sorted_df['end_cut']=''

#from IPython.core.debugger import Tracer

for index,row in grouped_clean_sorted_df.iterrows():
    
    future_index = index+1
    
    actual_plate = row['sitio_subida']
    actual_service = row['servicio_subida']
    actual_initial_hour = row['t_subidamin']
    actual_end_hour = row['t_subidamax']
        
    actual_id_expedicion = row['idExpedicion']
    
    if ((actual_plate!=past_plate)|(actual_service!=past_service)|(actual_id_expedicion!=past_id_expedicion)): #Assuming a pre-defined value, i.e., 30 seconds.
        start_cut = actual_initial_hour - (pd.to_datetime('2017-04-11 00:01:00') - pd.to_datetime('2017-04-11 00:00:30'))
    else:
        start_cut = actual_initial_hour - (actual_initial_hour - past_end_hour)/2
    
    
    if future_index >= len(grouped_clean_sorted_df.index): #We are at the end of the ddff. Assuming a pre-defined value, i.e., 30 seconds 
        end_cut = actual_end_hour + (pd.to_datetime('2017-04-11 00:01:00') - pd.to_datetime('2017-04-11 00:00:30'))
        
    else:        
        future_plate = grouped_clean_sorted_df.loc[future_index,'sitio_subida']
        future_service = grouped_clean_sorted_df.loc[future_index,'servicio_subida']
        future_initial_hour = grouped_clean_sorted_df.loc[future_index,'t_subidamin']
        future_end_hour = grouped_clean_sorted_df.loc[future_index,'t_subidamax']
        future_id_expedicion = grouped_clean_sorted_df.loc[future_index,'idExpedicion']
        
        if ((actual_plate!=future_plate)|(actual_service!=future_service)|(actual_id_expedicion!=future_id_expedicion)): #Changing service or plate... assuming a pre-defined value, i.e., 30 seconds
            end_cut = actual_end_hour + (pd.to_datetime('2017-04-11 00:01:00') - pd.to_datetime('2017-04-11 00:00:30'))
        else:
            end_cut = actual_end_hour + (future_initial_hour - actual_end_hour)/2
   
    past_initial_hour = actual_initial_hour
    past_end_hour = actual_end_hour
    past_id_expedicion = actual_id_expedicion
    past_plate = actual_plate
    past_service = actual_service
    grouped_clean_sorted_df.loc[index,'start_cut']=start_cut
    grouped_clean_sorted_df.loc[index,'end_cut']=end_cut

In [ ]:
grouped_clean_sorted_df.head()

In [ ]:
len(grouped_clean_sorted_df.index)

In [ ]:
example = example.reset_index(drop =True)

In [ ]:
for index,row in grouped_clean_sorted_df.iterrows():
    actual_plate = row['sitio_subida']
    actual_service = row['servicio_subida']
    start_cut = row['start_cut']
    end_cut = row['end_cut']
    
    actual_util_df = example[(example['PATENTE']==actual_plate)&(example['SERVICIO']==actual_service)&(start_cut<=example['TIEMPO'])&(example['TIEMPO']<=end_cut)]
    actual_util_df = actual_util_df[(actual_util_df['INGRESAN'] > actual_util_df['NO_VALIDAN'])] #Be aware of this condition
    actual_ev_obs = len(actual_util_df.index) #Mantains order of appearence
        
    grouped_clean_sorted_df.loc[index,'count_ev_obs'] = actual_ev_obs
    grouped_clean_sorted_df.loc[index,'ingresan'] = actual_util_df['INGRESAN'].sum()
    grouped_clean_sorted_df.loc[index,'no_validan'] = actual_util_df['NO_VALIDAN'].sum()
#    grouped_clean_sorted_df.loc[index,'tiempo_ev'] = actual_util_df['TIEMPO']

In [ ]:
grouped_clean_sorted_df.head()

In [ ]:
len(grouped_clean_sorted_df.index)

In [ ]:
processed_sorted_df[(processed_sorted_df['sitio_subida']=='BJFB62')&(processed_sorted_df['servicio_subida']=='507')&(processed_sorted_df['par_subida']=='T-18-156-OP-40')]

In [ ]:
grouped_clean_sorted_df['evasion_rate'] = grouped_clean_sorted_df['no_validan']/grouped_clean_sorted_df['ingresan']

In [ ]:
grouped_clean_sorted_df_test = grouped_clean_sorted_df[grouped_clean_sorted_df['diferencia_tiempo_secsmean'].notnull()]

In [ ]:
grouped_clean_sorted_df_test_clean = grouped_clean_sorted_df_test[grouped_clean_sorted_df_test['evasion_rate'].notnull()]

In [ ]:
grouped_clean_sorted_df_test_clean.head()

In [ ]:
torniquetesDataPath = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/03_BUSESTORNIQUETE/Avance_Consolidado_v2.xlsx"
busesTorniquete_df = pd.read_excel(torniquetesDataPath)
busesTorniquete_df.columns=['sitio_subida','fecha_instalacion']

busesTorniquete_df['sitio_subida'] = busesTorniquete_df['sitio_subida'].str.replace("-", "")
busesTorniquete_df['sitio_subida'] = busesTorniquete_df['sitio_subida'].str.replace(" ", "")

In [ ]:
busesTorniquete_df.head()

In [ ]:
#merged_turnstiles_df = pd.merge(grouped_clean_sorted_df_test_clean,busesTorniquete_df, on='sitio_subida', how='left')
#checking_missing = pd.isnull(merged_turnstiles_df['fecha_instalacion'])
#print('Not found in turnstile database: ' + str(sum(checking_missing))) #Without turnstiles, then NaT.
#merged_turnstiles_df['fecha_instalacion'] = pd.to_datetime(merged_turnstiles_df.fecha_instalacion)

In [ ]:
#grouped_clean_sorted_df_test_clean_noturns = merged_turnstiles_df[merged_turnstiles_df['fecha_instalacion'].isnull()]

In [ ]:
#grouped_clean_sorted_df_test_clean_noturns.head()

In [ ]:
fig_1, ax_1 = plt.subplots()

ax_1.scatter(grouped_clean_sorted_df_test_clean_noturns['diferencia_tiempo_secsmean'],grouped_clean_sorted_df_test_clean_noturns['evasion_rate'])
#ax_1.set_title('Evasion vs. Mean Time - No Turns, By Bus and Service, Original')
#ax_1.set_xlabel('Mean Time Interval [s]')
#ax_1.set_ylabel('Evasion Rate [%]')

In [ ]:
* Processing evasion-ddbb before merging

evasion_paradero = clean_processed_evasion[clean_processed_evasion['TP']=='P']
evasion_paradero_first = evasion_paradero[evasion_paradero['N_PUERTA']==1]
print(type(clean_processed_evasion['SERVICIO'][0])) #should be a string

print('Number of rows in complete evasion database is: ' + str(len(clean_processed_evasion.index)))
print('Number of rows in evasion in paradero database is: ' + str(len(evasion_paradero.index)))
print('Number of rows in evasion in paradero by first door database is: ' + str(len(evasion_paradero_first.index)))

* Filtering dates not in common dates

common_dates = TransantiagoConstants.common_dates
common_dates_timestamp = [pd.to_datetime(x) for x in common_dates]
common_dates_evasion = evasion_paradero_first[evasion_paradero_first['FECHA'].isin(common_dates_timestamp)]
len(common_dates_evasion.index) #Remember to store the number.

* <font color='red'>Filtering for one specific common date and analyze it to refactor and debug <- this part should be omitted in the final script. Above methodology is general</font>

example = common_dates_evasion[common_dates_evasion['FECHA']==pd.to_datetime('2017-04-11')]

len(example.index)

from RunSilentlyDailyEtapasBuilder import RunSilentlyDailyEtapasBuilderClass

date = '2017-04-11'
etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
processed_sorted_df = etapas_builder.runProcessedProcess()

len(processed_sorted_df.index)

#processed_sorted_df = processed_sorted_df[processed_sorted_df['diferencia_tiempo_secs']<=20]

* Processing sorted_df before reducing complexity

processed_sorted_df['servicio_subida'] =  processed_sorted_df['servicio_subida'].str.replace('T','')
processed_sorted_df['servicio_subida'] =  processed_sorted_df['servicio_subida'].str.split(' ').str[0]

processed_sorted_df['sitio_subida'] = processed_sorted_df['sitio_subida'].str.replace("-", "")
processed_sorted_df['sitio_subida'] = processed_sorted_df['sitio_subida'].str.replace(" ", "")

* First: filter etapas by patentes and servicios surveyed in evasion ddbb so to reduce search space

patentes = example['PATENTE'].unique()
patentes = patentes.tolist()
servicios = example['SERVICIO'].unique()
servicios = servicios.tolist()

clean_sorted_df = processed_sorted_df[(processed_sorted_df['sitio_subida'].isin(patentes))&(processed_sorted_df['servicio_subida'].isin(servicios))]

clean_sorted_df = clean_sorted_df.reset_index(drop=True)

len(clean_sorted_df.index)

* New methodology: appending evasion to trx

past_servicio = ''
past_patente = ''
past_time = pd.to_datetime('')
id_exp = 1
clean_sorted_df['idExpedicion'] = ''

for index,row in clean_sorted_df.iterrows():
    actual_servicio = row['servicio_subida']
    actual_patente = row['sitio_subida']
    actual_time = row['t_subida']
    
    #SOMETHING
    if((past_servicio==actual_servicio)&(actual_time - past_time <= (pd.to_datetime('2017-04-11 00:15:00')-pd.to_datetime('2017-04-11 00:00:00')))):
        clean_sorted_df.loc[index,'idExpedicion'] = id_exp

    elif((past_servicio==actual_servicio)&(actual_time - past_time > (pd.to_datetime('2017-04-11 00:15:00')-pd.to_datetime('2017-04-11 00:00:00')))):
        id_exp = id_exp + 1
        clean_sorted_df.loc[index,'idExpedicion'] = id_exp
        
    elif((past_servicio!=actual_servicio)):
        id_exp = 1
        clean_sorted_df.loc[index,'idExpedicion'] = id_exp
        

    past_servicio = actual_servicio
    past_patente = actual_patente
    past_time = actual_time
    

clean_sorted_df.head()

f = {'t_subida':['min', 'max', 'count'], 'diferencia_tiempo_secs':['mean']}
grouped_clean_sorted_df = clean_sorted_df.groupby(['sitio_subida','servicio_subida','idExpedicion','par_subida']).agg(f)
grouped_clean_sorted_df = grouped_clean_sorted_df.reset_index()
grouped_clean_sorted_df.columns = [''.join(col).strip() for col in grouped_clean_sorted_df.columns.values]
grouped_clean_sorted_df = grouped_clean_sorted_df.sort_values(by=['sitio_subida', 'servicio_subida', 'idExpedicion', 't_subidamin'], ascending=[True, True, True, True])
grouped_clean_sorted_df = grouped_clean_sorted_df.reset_index(drop=True)

i = 0
past_plate = '-'
past_service = '-'

past_par_subida = ['']

past_initial_hour = pd.to_datetime('2017-04-11 00:00:00')
past_end_hour = pd.to_datetime('2017-04-11 00:00:00')
past_id_expedicion = '-'

grouped_clean_sorted_df['start_cut']=''
grouped_clean_sorted_df['end_cut']=''

from IPython.core.debugger import Tracer

for index,row in grouped_clean_sorted_df.iterrows():
    
    future_index = index+1
    
    actual_plate = row['sitio_subida']
    actual_service = row['servicio_subida']
    actual_initial_hour = row['t_subidamin']
    actual_end_hour = row['t_subidamax']
        
    actual_id_expedicion = row['idExpedicion']
    
    if ((actual_plate!=past_plate)|(actual_service!=past_service)|(actual_id_expedicion!=past_id_expedicion)): #Assuming a pre-defined value, i.e., 30 seconds.
        start_cut = actual_initial_hour - (pd.to_datetime('2017-04-11 00:01:00') - pd.to_datetime('2017-04-11 00:00:30'))
    else:
        start_cut = actual_initial_hour - (actual_initial_hour - past_end_hour)/2
    
    
    if future_index >= len(grouped_clean_sorted_df.index): #We are at the end of the ddff. Assuming a pre-defined value, i.e., 30 seconds 
        end_cut = actual_end_hour + (pd.to_datetime('2017-04-11 00:01:00') - pd.to_datetime('2017-04-11 00:00:30'))
        
    else:        
        future_plate = grouped_clean_sorted_df.loc[future_index,'sitio_subida']
        future_service = grouped_clean_sorted_df.loc[future_index,'servicio_subida']
        future_initial_hour = grouped_clean_sorted_df.loc[future_index,'t_subidamin']
        future_end_hour = grouped_clean_sorted_df.loc[future_index,'t_subidamax']
        future_id_expedicion = grouped_clean_sorted_df.loc[future_index,'idExpedicion']
        
        if ((actual_plate!=future_plate)|(actual_service!=future_service)|(actual_id_expedicion!=future_id_expedicion)): #Changing service or plate... assuming a pre-defined value, i.e., 30 seconds
            end_cut = actual_end_hour + (pd.to_datetime('2017-04-11 00:01:00') - pd.to_datetime('2017-04-11 00:00:30'))
        else:
            end_cut = actual_end_hour + (future_initial_hour - actual_end_hour)/2
   
    past_initial_hour = actual_initial_hour
    past_end_hour = actual_end_hour
    past_id_expedicion = actual_id_expedicion
    past_plate = actual_plate
    past_service = actual_service
    grouped_clean_sorted_df.loc[index,'start_cut']=start_cut
    grouped_clean_sorted_df.loc[index,'end_cut']=end_cut

example.loc[:,'TIEMPO'] = example.loc[:,'FECHA'].dt.strftime('%Y-%m-%d') + ' ' + example.loc[:,'TIEMPO']
example['TIEMPO'] = pd.to_datetime(example['TIEMPO'])

example = example.reset_index(drop =True)

for index,row in grouped_clean_sorted_df.iterrows():
    actual_plate = row['sitio_subida']
    actual_service = row['servicio_subida']
    start_cut = row['start_cut']
    end_cut = row['end_cut']
    
    actual_util_df = example[(example['PATENTE']==actual_plate)&(example['SERVICIO']==actual_service)&(start_cut<=example['TIEMPO'])&(example['TIEMPO']<=end_cut)]
    actual_util_df = actual_util_df[(actual_util_df['INGRESAN'] > actual_util_df['NO_VALIDAN'])] #Be aware of this condition
    actual_ev_obs = len(actual_util_df.index) #Mantains order of appearence
        
    grouped_clean_sorted_df.loc[index,'count_ev_obs'] = actual_ev_obs
    grouped_clean_sorted_df.loc[index,'ingresan'] = actual_util_df['INGRESAN'].sum()
    grouped_clean_sorted_df.loc[index,'no_validan'] = actual_util_df['NO_VALIDAN'].sum()
#    grouped_clean_sorted_df.loc[index,'tiempo_ev'] = actual_util_df['TIEMPO']

grouped_clean_sorted_df.head()

len(grouped_clean_sorted_df.index)

processed_sorted_df[(processed_sorted_df['sitio_subida']=='BJFB62')&(processed_sorted_df['servicio_subida']=='507')&(processed_sorted_df['par_subida']=='T-18-156-OP-40')]

grouped_clean_sorted_df['evasion_rate'] = grouped_clean_sorted_df['no_validan']/grouped_clean_sorted_df['ingresan']

grouped_clean_sorted_df_test = grouped_clean_sorted_df[grouped_clean_sorted_df['diferencia_tiempo_secsmean'].notnull()]

grouped_clean_sorted_df_test_clean = grouped_clean_sorted_df_test[grouped_clean_sorted_df_test['evasion_rate'].notnull()]

grouped_clean_sorted_df_test_clean.head()

torniquetesDataPath = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/03_BUSESTORNIQUETE/Avance_Consolidado_v2.xlsx"
busesTorniquete_df = pd.read_excel(torniquetesDataPath)
busesTorniquete_df.columns=['sitio_subida','fecha_instalacion']

busesTorniquete_df['sitio_subida'] = busesTorniquete_df['sitio_subida'].str.replace("-", "")
busesTorniquete_df['sitio_subida'] = busesTorniquete_df['sitio_subida'].str.replace(" ", "")

busesTorniquete_df.head()

#merged_turnstiles_df = pd.merge(grouped_clean_sorted_df_test_clean,busesTorniquete_df, on='sitio_subida', how='left')
#checking_missing = pd.isnull(merged_turnstiles_df['fecha_instalacion'])
#print('Not found in turnstile database: ' + str(sum(checking_missing))) #Without turnstiles, then NaT.
#merged_turnstiles_df['fecha_instalacion'] = pd.to_datetime(merged_turnstiles_df.fecha_instalacion)

#grouped_clean_sorted_df_test_clean_noturns = merged_turnstiles_df[merged_turnstiles_df['fecha_instalacion'].isnull()]

#grouped_clean_sorted_df_test_clean_noturns.head()

fig_1, ax_1 = plt.subplots()

ax_1.scatter(grouped_clean_sorted_df_test_clean_noturns['diferencia_tiempo_secsmean'],grouped_clean_sorted_df_test_clean_noturns['evasion_rate'])
#ax_1.set_title('Evasion vs. Mean Time - No Turns, By Bus and Service, Original')
#ax_1.set_xlabel('Mean Time Interval [s]')
#ax_1.set_ylabel('Evasion Rate [%]')